In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist, fashion_mnist
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, AveragePooling2D
# from keras_visualizer import visualizer 
import tensorflow as tf

# Dataset

## Loading the Dataset

In [3]:
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()
print('Training Images', trainX.shape)
print('Training Labels', trainy.shape)
print('\nTesting Images', testX.shape)
print('Test Labels', testy.shape)

Training Images (60000, 28, 28)
Training Labels (60000,)

Testing Images (10000, 28, 28)
Test Labels (10000,)
